In [1]:
!pip install ultralytics
!pip install supervision

In [2]:
from ultralytics import YOLO

In [4]:
import pandas as pd
import os


merged_posts_data = pd.read_csv('filtered_mapped_posts_2k_per_inflCateg.csv')

merged_posts_data_no_duplicates = merged_posts_data.drop_duplicates()


# Construct the image file name and check if it exists in the directory
merged_posts_data_no_duplicates['constructed_file_name'] = merged_posts_data_no_duplicates['OwnerUsername'] + '-' + merged_posts_data_no_duplicates['Image_file_name'] 

# Change to the directory containing the images
os.chdir('images_output/output_images')

# List of available image files in the directory
available_images = set(os.listdir('.'))


# Navigate back to the root directory -- do it 2 times
os.chdir('..')
os.chdir('..')

# Get the current working directory
current_dir = os.getcwd()
print("Current working directory:", current_dir)


# Filter the DataFrame to only include rows with existing images
filtered_df = merged_posts_data_no_duplicates[merged_posts_data_no_duplicates['constructed_file_name'].isin(available_images)]

# Reset the index of the filtered DataFrame
filtered_df.reset_index(drop=True, inplace=True)

/tmp/ipykernel_15257/2685018602.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_posts_data_no_duplicates['constructed_file_name'] = merged_posts_data_no_duplicates['OwnerUsername'] + '-' + merged_posts_data_no_duplicates['Image_file_name']


Current working directory: /home/jovyan


In [5]:
# Directory containing your images
image_dir = 'images_output/output_images'

img_paths = [os.path.join(image_dir, filtered_df.iloc[idx]['constructed_file_name']) for idx in range(len(filtered_df))]
  
  

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from ultralytics import YOLO
import os
import csv
import contextlib

# Load the YOLO model
yolo = YOLO('yolov8x.pt')


# Define the computation device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
yolo.to(device)


class ImageDataset(Dataset):
    def __init__(self, image_dir,img_paths, transform=None):
        self.image_dir = image_dir
        self.image_files = os.listdir(image_dir)
        self.transform = transform
        self.img_paths = img_paths 

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        
        #img_paths = [os.path.join(image_dir, filtered_df.iloc[idx]['constructed_file_name']) for idx in range(len(filtered_df))]
  
        image = Image.open(self.img_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image) #.unsqueeze(0).to(device)  # Add batch dimension and move to device


        return image, img_name

# Define the torchvision image transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),  # Converts image to PyTorch tensor and scales pixel values to [0, 1]
])

# Create the dataset and dataloader
image_dir = 'images_output/output_images'
dataset = ImageDataset(image_dir, img_paths,  transform=transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=False, num_workers=4)

# Open the CSV file in write mode initially to write headers
csv_file = 'yolo_detection_results_06.csv'
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['image_name', 'box_x1', 'box_y1', 'box_x2', 'box_y2', 'confidence', 'class'])

# Function to perform inference and process results
def process_batch(images, img_names):
    images = images.to(device)
    #with open(os.devnull, 'w') as f, contextlib.redirect_stdout(f):
        #results = yolo.predict(source=images, save=True, imgsz=320, conf=0.4)  # Suppress output

    results = yolo.predict(source=images, save=False, imgsz=320, conf=0.6 ,verbose=False)  #yolo(image_tensor)

    #yolo(images)

    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        for i, result in enumerate(results):
            boxes = result.boxes.xyxy.cpu().numpy()
            labels = result.boxes.cls.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()
            img_name = img_names[i]

            for box, label, confidence in zip(boxes, labels, confidences):
                x1, y1, x2, y2 = box
                writer.writerow([img_name, x1, y1, x2, y2, confidence, label])
            
            #print(f"Processed {img_name} with {len(boxes)} objects detected.")

# Iterate through the dataloader and process images
for batch in dataloader:
    images, img_names = batch
    process_batch(images, img_names)

print(f"Process complete.")

Process complete.


In [12]:
import pandas as pd

pd.read_csv('Rcnn_detection_results.csv')

,image_name,box_x1,box_y1,box_x2,box_y2,confidence,class
0,0ldisme-1638622741936691543.jpg,15.783750,127.595330,58.602005,194.57515,0.891238,86
1,0ldisme-1638622741936691543.jpg,114.156425,143.370200,208.744230,255.45355,0.873835,84
2,0ldisme-1638622741936691543.jpg,58.461845,144.566360,139.983640,254.37383,0.859680,84
3,0ldisme-1638622741936691543.jpg,35.904810,8.719462,138.499820,111.54178,0.734970,84
4,0ldisme-1638622741936691543.jpg,92.414140,226.373290,111.384470,256.00000,0.658372,1
...,...,...,...,...,...,...,...
2124688,zyndl_lala-1720548924400637702.jpg,19.174664,122.219180,131.365190,255.77086,0.997754,1
2124689,zyndl_lala-1720548924400637702.jpg,38.303900,146.095900,53.554760,175.40134,0.909581,1
2124690,zyndl_lala-1720548924400637702.jpg,1.996982,138.595200,250.618520,256.00000,0.870160,7
2124691,zyndl_lala-1720548924400637702.jpg,22.238085,146.505920,38.692547,176.16072,0.827318,1


In [8]:
import pandas as pd

pd.read_csv('yolo_detection_results_06.csv')

,image_name,box_x1,box_y1,box_x2,box_y2,confidence,class
0,pluswonderland-1605961109646837370.jpg,163.789400,78.816605,232.715670,159.556840,0.700889,67.0
1,pluswonderland-1605961109646837370.jpg,52.945070,34.979416,120.449600,109.886215,0.649784,65.0
2,leticiabuffonn-1631427819249464936.jpg,77.355350,15.120434,218.617520,126.653854,0.835240,62.0
3,mollysport-1781406177155308410.jpg,112.144800,0.325729,255.302400,92.801796,0.683709,73.0
4,mollysport-1781406177155308410.jpg,0.039457,1.016327,25.387787,205.755130,0.617782,73.0
...,...,...,...,...,...,...,...
2530328,batonrougemoms-1726452539391605301.jpg,7.191040,165.189210,228.443070,255.704960,0.795254,0.0
2530329,valentina_luchetta-1845045138066631581.jpg,19.906693,122.738890,132.221100,255.695650,0.927382,0.0
2530330,valentina_luchetta-1845045138066631581.jpg,21.004086,146.355590,40.049560,176.996340,0.612005,0.0
2530331,allstuffnice-1814374200184304322.jpg,0.647919,0.626541,254.746730,255.259780,0.954271,0.0
